In [1]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 46.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from ktrain.text.zsl import ZeroShotClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Find  first n categories from the prpbability

In [4]:
def find_top_categories_for_lists(lists_of_category_scores, num_categories):
    top_categories = []
    for category_scores in lists_of_category_scores:
        # Sort the categories based on their values in descending order
        sorted_categories = sorted(category_scores, key=lambda x: x[1], reverse=True)
        # Select the top 'num_categories' categories
        top_categories_list = [category[0] for category in sorted_categories[:num_categories]]
        top_categories.append(top_categories_list)
    return top_categories

# Find the maximum first n categories from the probability

In [5]:
def find_matching_category_or_first(list1, list2):
    for category in list1:
        if category in list2:
            return category
    return list1[0]
def find_matching_category_for_each_list(list1, list2):
    results = []
    for sublist, category in zip(list1, list2):
        result = find_matching_category_or_first(sublist, [category])
        results.append(result)
    return results

In [6]:
def compute_classification_report(y_true, y_pred):

    report = classification_report(y_true, y_pred)
    print(report)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [7]:
def find_max_value(category_scores):
    max_value = max(category_scores, key=lambda x: x[1])[1]
    return max_value

In [8]:
df = pd.read_csv("https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/TOPICS/DATA/News_Category_Dataset_v3/News_Category_Dataset_v3.csv")
df

,Unnamed: 0,link,headline,category,short_description,authors,date
0,0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...,...
209521,209521,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209522,209522,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",NaN,2012-01-28
209523,209523,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",NaN,2012-01-28
209524,209524,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,NaN,2012-01-28


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209526 entries, 0 to 209525
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         209526 non-null  int64 
 1   link               209526 non-null  object
 2   headline           209520 non-null  object
 3   category           209526 non-null  object
 4   short_description  189814 non-null  object
 5   authors            172108 non-null  object
 6   date               209526 non-null  object
dtypes: int64(1), object(6)
memory usage: 11.2+ MB


In [10]:
df.dropna(subset=['short_description'], inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189814 entries, 0 to 209525
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         189814 non-null  int64 
 1   link               189814 non-null  object
 2   headline           189813 non-null  object
 3   category           189814 non-null  object
 4   short_description  189814 non-null  object
 5   authors            156859 non-null  object
 6   date               189814 non-null  object
dtypes: int64(1), object(6)
memory usage: 11.6+ MB


In [12]:
X = df.short_description
Y = df.category

In [13]:
np.unique(Y)

array(['ARTS', 'ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE',
       'COMEDY', 'CRIME', 'CULTURE & ARTS', 'DIVORCE', 'EDUCATION',
       'ENTERTAINMENT', 'ENVIRONMENT', 'FIFTY', 'FOOD & DRINK',
       'GOOD NEWS', 'GREEN', 'HEALTHY LIVING', 'HOME & LIVING', 'IMPACT',
       'LATINO VOICES', 'MEDIA', 'MONEY', 'PARENTING', 'PARENTS',
       'POLITICS', 'QUEER VOICES', 'RELIGION', 'SCIENCE', 'SPORTS',
       'STYLE', 'STYLE & BEAUTY', 'TASTE', 'TECH', 'THE WORLDPOST',
       'TRAVEL', 'U.S. NEWS', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS',
       'WOMEN', 'WORLD NEWS', 'WORLDPOST'], dtype=object)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,Y , shuffle=True, test_size = 100, random_state = 42 , stratify=Y)

In [15]:
np.unique(y_test)

array(['ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COMEDY', 'CRIME',
       'CULTURE & ARTS', 'DIVORCE', 'ENTERTAINMENT', 'ENVIRONMENT',
       'FIFTY', 'FOOD & DRINK', 'GOOD NEWS', 'GREEN', 'HEALTHY LIVING',
       'HOME & LIVING', 'IMPACT', 'MEDIA', 'MONEY', 'PARENTING',
       'PARENTS', 'POLITICS', 'QUEER VOICES', 'RELIGION', 'SCIENCE',
       'SPORTS', 'STYLE', 'STYLE & BEAUTY', 'TASTE', 'TECH',
       'THE WORLDPOST', 'TRAVEL', 'U.S. NEWS', 'WEDDINGS', 'WEIRD NEWS',
       'WELLNESS', 'WOMEN', 'WORLD NEWS', 'WORLDPOST'], dtype=object)

In [16]:
zsl = ZeroShotClassifier()
labels=['ARTS', 'ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE',
       'COMEDY', 'CRIME', 'CULTURE & ARTS', 'DIVORCE', 'EDUCATION',
       'ENTERTAINMENT', 'ENVIRONMENT', 'FIFTY', 'FOOD & DRINK',
       'GOOD NEWS', 'GREEN', 'HEALTHY LIVING', 'HOME & LIVING', 'IMPACT',
       'LATINO VOICES', 'MEDIA', 'MONEY', 'PARENTING', 'PARENTS',
       'POLITICS', 'QUEER VOICES', 'RELIGION', 'SCIENCE', 'SPORTS',
       'STYLE', 'STYLE & BEAUTY', 'TASTE', 'TECH', 'THE WORLDPOST',
       'TRAVEL', 'U.S. NEWS', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS',
       'WOMEN', 'WORLD NEWS', 'WORLDPOST']

In [17]:
%%time
doc_prb = zsl.predict(X_test.values, labels=labels, include_labels=True , batch_size = 16)

CPU times: user 39.9 s, sys: 208 ms, total: 40.1 s
Wall time: 42.6 s


In [18]:
doc_prb

[[('ARTS', 0.1188773512840271),
  ('ARTS & CULTURE', 0.1186971366405487),
  ('BLACK VOICES', 0.07202143967151642),
  ('BUSINESS', 0.0012244608951732516),
  ('COLLEGE', 0.07074254751205444),
  ('COMEDY', 0.0059710340574383736),
  ('CRIME', 0.003896599868312478),
  ('CULTURE & ARTS', 0.06370508670806885),
  ('DIVORCE', 0.00313295004889369),
  ('EDUCATION', 0.0070751989260315895),
  ('ENTERTAINMENT', 0.041494812816381454),
  ('ENVIRONMENT', 0.015150044113397598),
  ('FIFTY', 0.05532555282115936),
  ('FOOD & DRINK', 0.0014343183720484376),
  ('GOOD NEWS', 0.2501695454120636),
  ('GREEN', 0.13531526923179626),
  ('HEALTHY LIVING', 0.04809567704796791),
  ('HOME & LIVING', 0.02907346561551094),
  ('IMPACT', 0.9145495295524597),
  ('LATINO VOICES', 0.2991216778755188),
  ('MEDIA', 0.11704033613204956),
  ('MONEY', 0.0017474949127063155),
  ('PARENTING', 0.012625418603420258),
  ('PARENTS', 0.009756949730217457),
  ('POLITICS', 0.0036491346545517445),
  ('QUEER VOICES', 0.11919135600328445),
 

# **1**

In [19]:
df2_pred1 = find_top_categories_for_lists(doc_prb , 1)
# y_trn_pred = find_max_categories_for_lists(y_trn_prb)
# y_tst_pred = find_max_categories_for_lists(y_tst_prb)

In [20]:
df2_pred1

[['IMPACT'],
 ['WORLDPOST'],
 ['POLITICS'],
 ['WOMEN'],
 ['IMPACT'],
 ['ARTS'],
 ['COMEDY'],
 ['IMPACT'],
 ['MONEY'],
 ['IMPACT'],
 ['ENTERTAINMENT'],
 ['MEDIA'],
 ['MEDIA'],
 ['IMPACT'],
 ['MONEY'],
 ['WEIRD NEWS'],
 ['SCIENCE'],
 ['POLITICS'],
 ['MONEY'],
 ['IMPACT'],
 ['STYLE'],
 ['WORLDPOST'],
 ['IMPACT'],
 ['IMPACT'],
 ['MONEY'],
 ['FIFTY'],
 ['WEDDINGS'],
 ['IMPACT'],
 ['WORLDPOST'],
 ['IMPACT'],
 ['ENTERTAINMENT'],
 ['IMPACT'],
 ['IMPACT'],
 ['IMPACT'],
 ['IMPACT'],
 ['MEDIA'],
 ['PARENTING'],
 ['ENTERTAINMENT'],
 ['DIVORCE'],
 ['LATINO VOICES'],
 ['COLLEGE'],
 ['WEIRD NEWS'],
 ['IMPACT'],
 ['IMPACT'],
 ['IMPACT'],
 ['TRAVEL'],
 ['FOOD & DRINK'],
 ['POLITICS'],
 ['IMPACT'],
 ['POLITICS'],
 ['RELIGION'],
 ['IMPACT'],
 ['IMPACT'],
 ['STYLE'],
 ['ARTS'],
 ['IMPACT'],
 ['IMPACT'],
 ['IMPACT'],
 ['COMEDY'],
 ['IMPACT'],
 ['IMPACT'],
 ['IMPACT'],
 ['MONEY'],
 ['IMPACT'],
 ['IMPACT'],
 ['IMPACT'],
 ['COLLEGE'],
 ['RELIGION'],
 ['CRIME'],
 ['MEDIA'],
 ['CRIME'],
 ['WELLNESS'],
 ['ENTERT

In [21]:
compute_classification_report(y_test , df2_pred1)

                precision    recall  f1-score   support

          ARTS       0.00      0.00      0.00         0
ARTS & CULTURE       0.00      0.00      0.00         1
  BLACK VOICES       0.00      0.00      0.00         2
      BUSINESS       0.00      0.00      0.00         3
       COLLEGE       0.00      0.00      0.00         0
        COMEDY       0.00      0.00      0.00         2
         CRIME       0.00      0.00      0.00         1
CULTURE & ARTS       0.00      0.00      0.00         1
       DIVORCE       1.00      1.00      1.00         2
     EDUCATION       0.00      0.00      0.00         0
 ENTERTAINMENT       0.20      0.12      0.15         8
   ENVIRONMENT       0.00      0.00      0.00         1
         FIFTY       0.00      0.00      0.00         1
  FOOD & DRINK       1.00      0.67      0.80         3
     GOOD NEWS       0.00      0.00      0.00         1
         GREEN       0.00      0.00      0.00         1
HEALTHY LIVING       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

0.19

In [22]:
# compute_classification_report(y_trn , y_trn_pred)

In [23]:
# compute_classification_report(y_tst , y_tst_pred)

# **2**

## After getting the 2 maximum probability categories i check if **the first or the second** category is the same as true label and get the classification report

In [24]:
df2_pred2 = find_top_categories_for_lists(doc_prb , 2)
# y_trn_pred = find_max_2_categories_for_lists(y_trn_prb)
# y_tst_pred = find_max_2_categories_for_lists(y_tst_prb)

In [25]:
df2_pred2

[['IMPACT', 'TRAVEL'],
 ['WORLDPOST', 'IMPACT'],
 ['POLITICS', 'WEIRD NEWS'],
 ['WOMEN', 'WEIRD NEWS'],
 ['IMPACT', 'MONEY'],
 ['ARTS', 'CULTURE & ARTS'],
 ['COMEDY', 'WOMEN'],
 ['IMPACT', 'SPORTS'],
 ['MONEY', 'TASTE'],
 ['IMPACT', 'CRIME'],
 ['ENTERTAINMENT', 'FOOD & DRINK'],
 ['MEDIA', 'WORLDPOST'],
 ['MEDIA', 'TECH'],
 ['IMPACT', 'WEIRD NEWS'],
 ['MONEY', 'IMPACT'],
 ['WEIRD NEWS', 'CRIME'],
 ['SCIENCE', 'IMPACT'],
 ['POLITICS', 'IMPACT'],
 ['MONEY', 'BUSINESS'],
 ['IMPACT', 'CRIME'],
 ['STYLE', 'COLLEGE'],
 ['WORLDPOST', 'IMPACT'],
 ['IMPACT', 'POLITICS'],
 ['IMPACT', 'FOOD & DRINK'],
 ['MONEY', 'IMPACT'],
 ['FIFTY', 'IMPACT'],
 ['WEDDINGS', 'IMPACT'],
 ['IMPACT', 'U.S. NEWS'],
 ['WORLDPOST', 'THE WORLDPOST'],
 ['IMPACT', 'WORLDPOST'],
 ['ENTERTAINMENT', 'MEDIA'],
 ['IMPACT', 'U.S. NEWS'],
 ['IMPACT', 'MONEY'],
 ['IMPACT', 'GOOD NEWS'],
 ['IMPACT', 'U.S. NEWS'],
 ['MEDIA', 'IMPACT'],
 ['PARENTING', 'IMPACT'],
 ['ENTERTAINMENT', 'WOMEN'],
 ['DIVORCE', 'PARENTING'],
 ['LATINO VOICES

In [26]:
df2_pred2 = find_matching_category_for_each_list(df2_pred2 , y_test)
# y_trn_pred = return_best_category_from_2_categories(y_trn , y_trn_pred)
# y_tst_pred = return_best_category_from_2_categories(y_tst , y_tst_pred)

In [27]:
df2_pred2

['IMPACT',
 'WORLDPOST',
 'POLITICS',
 'WOMEN',
 'IMPACT',
 'CULTURE & ARTS',
 'COMEDY',
 'IMPACT',
 'MONEY',
 'IMPACT',
 'ENTERTAINMENT',
 'MEDIA',
 'MEDIA',
 'IMPACT',
 'MONEY',
 'WEIRD NEWS',
 'SCIENCE',
 'POLITICS',
 'BUSINESS',
 'IMPACT',
 'STYLE',
 'WORLDPOST',
 'POLITICS',
 'IMPACT',
 'MONEY',
 'FIFTY',
 'WEDDINGS',
 'IMPACT',
 'WORLDPOST',
 'IMPACT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'MEDIA',
 'PARENTING',
 'ENTERTAINMENT',
 'DIVORCE',
 'LATINO VOICES',
 'COLLEGE',
 'WEIRD NEWS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'TRAVEL',
 'FOOD & DRINK',
 'POLITICS',
 'IMPACT',
 'POLITICS',
 'RELIGION',
 'IMPACT',
 'IMPACT',
 'STYLE',
 'ENTERTAINMENT',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'COMEDY',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'MONEY',
 'POLITICS',
 'IMPACT',
 'IMPACT',
 'COLLEGE',
 'RELIGION',
 'CRIME',
 'MEDIA',
 'CRIME',
 'WELLNESS',
 'ENTERTAINMENT',
 'WELLNESS',
 'IMPACT',
 'FOOD & DRINK',
 'MEDIA',
 'IMPACT',
 'WORLDPOST',
 'IMPACT',
 'MEDIA',
 'CULTURE

In [28]:
compute_classification_report(y_test , df2_pred2)

                precision    recall  f1-score   support

          ARTS       0.00      0.00      0.00         0
ARTS & CULTURE       0.00      0.00      0.00         1
  BLACK VOICES       0.00      0.00      0.00         2
      BUSINESS       1.00      0.33      0.50         3
       COLLEGE       0.00      0.00      0.00         0
        COMEDY       0.00      0.00      0.00         2
         CRIME       0.00      0.00      0.00         1
CULTURE & ARTS       0.50      1.00      0.67         1
       DIVORCE       1.00      1.00      1.00         2
     EDUCATION       0.00      0.00      0.00         0
 ENTERTAINMENT       0.40      0.25      0.31         8
   ENVIRONMENT       0.00      0.00      0.00         1
         FIFTY       0.00      0.00      0.00         1
  FOOD & DRINK       1.00      0.67      0.80         3
     GOOD NEWS       0.00      0.00      0.00         1
         GREEN       0.00      0.00      0.00         1
HEALTHY LIVING       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

0.29

In [29]:
# compute_classification_report(y_trn , y_trn_pred)

In [30]:
# compute_classification_report(y_tst , y_tst_pred)

# **3**

In [31]:
df2_pred3 = find_top_categories_for_lists(doc_prb , 3)


In [32]:
df2_pred3

[['IMPACT', 'TRAVEL', 'WELLNESS'],
 ['WORLDPOST', 'IMPACT', 'THE WORLDPOST'],
 ['POLITICS', 'WEIRD NEWS', 'IMPACT'],
 ['WOMEN', 'WEIRD NEWS', 'IMPACT'],
 ['IMPACT', 'MONEY', 'BUSINESS'],
 ['ARTS', 'CULTURE & ARTS', 'ARTS & CULTURE'],
 ['COMEDY', 'WOMEN', 'WEIRD NEWS'],
 ['IMPACT', 'SPORTS', 'COLLEGE'],
 ['MONEY', 'TASTE', 'FOOD & DRINK'],
 ['IMPACT', 'CRIME', 'POLITICS'],
 ['ENTERTAINMENT', 'FOOD & DRINK', 'MEDIA'],
 ['MEDIA', 'WORLDPOST', 'THE WORLDPOST'],
 ['MEDIA', 'TECH', 'IMPACT'],
 ['IMPACT', 'WEIRD NEWS', 'DIVORCE'],
 ['MONEY', 'IMPACT', 'TRAVEL'],
 ['WEIRD NEWS', 'CRIME', 'POLITICS'],
 ['SCIENCE', 'IMPACT', 'WEIRD NEWS'],
 ['POLITICS', 'IMPACT', 'THE WORLDPOST'],
 ['MONEY', 'BUSINESS', 'CRIME'],
 ['IMPACT', 'CRIME', 'U.S. NEWS'],
 ['STYLE', 'COLLEGE', 'IMPACT'],
 ['WORLDPOST', 'IMPACT', 'HOME & LIVING'],
 ['IMPACT', 'POLITICS', 'U.S. NEWS'],
 ['IMPACT', 'FOOD & DRINK', 'TASTE'],
 ['MONEY', 'IMPACT', 'TECH'],
 ['FIFTY', 'IMPACT', 'WORLDPOST'],
 ['WEDDINGS', 'IMPACT', 'DIVORCE'],

In [33]:
df2_pred3 = find_matching_category_for_each_list(df2_pred3 , y_test)


In [34]:
df2_pred3

['IMPACT',
 'WORLDPOST',
 'POLITICS',
 'WOMEN',
 'IMPACT',
 'CULTURE & ARTS',
 'COMEDY',
 'IMPACT',
 'FOOD & DRINK',
 'POLITICS',
 'ENTERTAINMENT',
 'MEDIA',
 'MEDIA',
 'IMPACT',
 'TRAVEL',
 'WEIRD NEWS',
 'SCIENCE',
 'POLITICS',
 'BUSINESS',
 'IMPACT',
 'STYLE',
 'WORLDPOST',
 'POLITICS',
 'IMPACT',
 'MONEY',
 'FIFTY',
 'WEDDINGS',
 'IMPACT',
 'WORLDPOST',
 'IMPACT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'MEDIA',
 'PARENTING',
 'ENTERTAINMENT',
 'DIVORCE',
 'LATINO VOICES',
 'COLLEGE',
 'WEIRD NEWS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'TRAVEL',
 'FOOD & DRINK',
 'POLITICS',
 'IMPACT',
 'POLITICS',
 'RELIGION',
 'IMPACT',
 'IMPACT',
 'STYLE',
 'ENTERTAINMENT',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'COMEDY',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'MONEY',
 'POLITICS',
 'IMPACT',
 'IMPACT',
 'COLLEGE',
 'RELIGION',
 'CRIME',
 'MEDIA',
 'CRIME',
 'WELLNESS',
 'ENTERTAINMENT',
 'WELLNESS',
 'WORLDPOST',
 'FOOD & DRINK',
 'MEDIA',
 'IMPACT',
 'WORLDPOST',
 'IMPACT',
 'MEDI

In [35]:
compute_classification_report(y_test , df2_pred3)

                precision    recall  f1-score   support

          ARTS       0.00      0.00      0.00         0
ARTS & CULTURE       0.00      0.00      0.00         1
  BLACK VOICES       0.00      0.00      0.00         2
      BUSINESS       1.00      0.33      0.50         3
       COLLEGE       0.00      0.00      0.00         0
        COMEDY       0.00      0.00      0.00         2
         CRIME       0.00      0.00      0.00         1
CULTURE & ARTS       0.50      1.00      0.67         1
       DIVORCE       1.00      1.00      1.00         2
     EDUCATION       0.00      0.00      0.00         0
 ENTERTAINMENT       0.40      0.25      0.31         8
   ENVIRONMENT       0.00      0.00      0.00         1
         FIFTY       0.00      0.00      0.00         1
  FOOD & DRINK       1.00      1.00      1.00         3
     GOOD NEWS       0.00      0.00      0.00         1
         GREEN       0.00      0.00      0.00         1
HEALTHY LIVING       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

0.33

# **4**

In [36]:
df2_pred4 = find_top_categories_for_lists(doc_prb , 4)


In [37]:
df2_pred4

[['IMPACT', 'TRAVEL', 'WELLNESS', 'SPORTS'],
 ['WORLDPOST', 'IMPACT', 'THE WORLDPOST', 'MONEY'],
 ['POLITICS', 'WEIRD NEWS', 'IMPACT', 'U.S. NEWS'],
 ['WOMEN', 'WEIRD NEWS', 'IMPACT', 'WORLDPOST'],
 ['IMPACT', 'MONEY', 'BUSINESS', 'ENVIRONMENT'],
 ['ARTS', 'CULTURE & ARTS', 'ARTS & CULTURE', 'MEDIA'],
 ['COMEDY', 'WOMEN', 'WEIRD NEWS', 'ENTERTAINMENT'],
 ['IMPACT', 'SPORTS', 'COLLEGE', 'QUEER VOICES'],
 ['MONEY', 'TASTE', 'FOOD & DRINK', 'IMPACT'],
 ['IMPACT', 'CRIME', 'POLITICS', 'U.S. NEWS'],
 ['ENTERTAINMENT', 'FOOD & DRINK', 'MEDIA', 'ARTS'],
 ['MEDIA', 'WORLDPOST', 'THE WORLDPOST', 'U.S. NEWS'],
 ['MEDIA', 'TECH', 'IMPACT', 'DIVORCE'],
 ['IMPACT', 'WEIRD NEWS', 'DIVORCE', 'U.S. NEWS'],
 ['MONEY', 'IMPACT', 'TRAVEL', 'BUSINESS'],
 ['WEIRD NEWS', 'CRIME', 'POLITICS', 'IMPACT'],
 ['SCIENCE', 'IMPACT', 'WEIRD NEWS', 'ENVIRONMENT'],
 ['POLITICS', 'IMPACT', 'THE WORLDPOST', 'U.S. NEWS'],
 ['MONEY', 'BUSINESS', 'CRIME', 'IMPACT'],
 ['IMPACT', 'CRIME', 'U.S. NEWS', 'WORLDPOST'],
 ['STYLE'

In [38]:
df2_pred4 = find_matching_category_for_each_list(df2_pred4 , y_test)


In [39]:
df2_pred4

['IMPACT',
 'WORLDPOST',
 'POLITICS',
 'WOMEN',
 'IMPACT',
 'CULTURE & ARTS',
 'ENTERTAINMENT',
 'IMPACT',
 'FOOD & DRINK',
 'POLITICS',
 'ENTERTAINMENT',
 'MEDIA',
 'MEDIA',
 'IMPACT',
 'TRAVEL',
 'WEIRD NEWS',
 'SCIENCE',
 'POLITICS',
 'BUSINESS',
 'IMPACT',
 'STYLE & BEAUTY',
 'WORLDPOST',
 'POLITICS',
 'IMPACT',
 'MONEY',
 'FIFTY',
 'WEDDINGS',
 'IMPACT',
 'WORLDPOST',
 'IMPACT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'ENTERTAINMENT',
 'PARENTING',
 'ENTERTAINMENT',
 'DIVORCE',
 'LATINO VOICES',
 'COLLEGE',
 'POLITICS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'TRAVEL',
 'FOOD & DRINK',
 'POLITICS',
 'IMPACT',
 'POLITICS',
 'RELIGION',
 'IMPACT',
 'IMPACT',
 'STYLE & BEAUTY',
 'ENTERTAINMENT',
 'STYLE & BEAUTY',
 'IMPACT',
 'IMPACT',
 'COMEDY',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'MONEY',
 'POLITICS',
 'IMPACT',
 'IMPACT',
 'COLLEGE',
 'RELIGION',
 'CRIME',
 'POLITICS',
 'CRIME',
 'WELLNESS',
 'ENTERTAINMENT',
 'WELLNESS',
 'WORLDPOST',
 'FOOD & DRINK',
 'MEDIA',


In [40]:
compute_classification_report(y_test , df2_pred4)

                precision    recall  f1-score   support

          ARTS       0.00      0.00      0.00         0
ARTS & CULTURE       0.00      0.00      0.00         1
  BLACK VOICES       0.00      0.00      0.00         2
      BUSINESS       1.00      0.33      0.50         3
       COLLEGE       0.00      0.00      0.00         0
        COMEDY       0.00      0.00      0.00         2
         CRIME       0.33      1.00      0.50         1
CULTURE & ARTS       0.50      1.00      0.67         1
       DIVORCE       1.00      1.00      1.00         2
     EDUCATION       0.00      0.00      0.00         0
 ENTERTAINMENT       0.57      0.50      0.53         8
   ENVIRONMENT       0.00      0.00      0.00         1
         FIFTY       0.00      0.00      0.00         1
  FOOD & DRINK       1.00      1.00      1.00         3
     GOOD NEWS       0.00      0.00      0.00         1
         GREEN       0.00      0.00      0.00         1
HEALTHY LIVING       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

0.42

# **5**

In [41]:
df2_pred5 = find_top_categories_for_lists(doc_prb , 5)


In [42]:
df2_pred5

[['IMPACT', 'TRAVEL', 'WELLNESS', 'SPORTS', 'LATINO VOICES'],
 ['WORLDPOST', 'IMPACT', 'THE WORLDPOST', 'MONEY', 'WEIRD NEWS'],
 ['POLITICS', 'WEIRD NEWS', 'IMPACT', 'U.S. NEWS', 'QUEER VOICES'],
 ['WOMEN', 'WEIRD NEWS', 'IMPACT', 'WORLDPOST', 'CRIME'],
 ['IMPACT', 'MONEY', 'BUSINESS', 'ENVIRONMENT', 'WORLD NEWS'],
 ['ARTS', 'CULTURE & ARTS', 'ARTS & CULTURE', 'MEDIA', 'IMPACT'],
 ['COMEDY', 'WOMEN', 'WEIRD NEWS', 'ENTERTAINMENT', 'MEDIA'],
 ['IMPACT', 'SPORTS', 'COLLEGE', 'QUEER VOICES', 'U.S. NEWS'],
 ['MONEY', 'TASTE', 'FOOD & DRINK', 'IMPACT', 'FIFTY'],
 ['IMPACT', 'CRIME', 'POLITICS', 'U.S. NEWS', 'PARENTING'],
 ['ENTERTAINMENT', 'FOOD & DRINK', 'MEDIA', 'ARTS', 'IMPACT'],
 ['MEDIA', 'WORLDPOST', 'THE WORLDPOST', 'U.S. NEWS', 'IMPACT'],
 ['MEDIA', 'TECH', 'IMPACT', 'DIVORCE', 'BLACK VOICES'],
 ['IMPACT', 'WEIRD NEWS', 'DIVORCE', 'U.S. NEWS', 'POLITICS'],
 ['MONEY', 'IMPACT', 'TRAVEL', 'BUSINESS', 'WEIRD NEWS'],
 ['WEIRD NEWS', 'CRIME', 'POLITICS', 'IMPACT', 'U.S. NEWS'],
 ['SCIENC

In [43]:
df2_pred5 = find_matching_category_for_each_list(df2_pred5 , y_test)


In [44]:
df2_pred5

['IMPACT',
 'WORLDPOST',
 'POLITICS',
 'WOMEN',
 'IMPACT',
 'CULTURE & ARTS',
 'ENTERTAINMENT',
 'IMPACT',
 'FOOD & DRINK',
 'POLITICS',
 'ENTERTAINMENT',
 'MEDIA',
 'MEDIA',
 'POLITICS',
 'TRAVEL',
 'WEIRD NEWS',
 'SCIENCE',
 'POLITICS',
 'BUSINESS',
 'IMPACT',
 'STYLE & BEAUTY',
 'HEALTHY LIVING',
 'POLITICS',
 'IMPACT',
 'MONEY',
 'FIFTY',
 'WEDDINGS',
 'IMPACT',
 'WORLDPOST',
 'IMPACT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'ENTERTAINMENT',
 'PARENTING',
 'ENTERTAINMENT',
 'DIVORCE',
 'LATINO VOICES',
 'COLLEGE',
 'POLITICS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'TRAVEL',
 'FOOD & DRINK',
 'POLITICS',
 'IMPACT',
 'POLITICS',
 'RELIGION',
 'THE WORLDPOST',
 'ENVIRONMENT',
 'STYLE & BEAUTY',
 'ENTERTAINMENT',
 'STYLE & BEAUTY',
 'IMPACT',
 'IMPACT',
 'COMEDY',
 'IMPACT',
 'SPORTS',
 'IMPACT',
 'MONEY',
 'POLITICS',
 'IMPACT',
 'IMPACT',
 'COLLEGE',
 'TRAVEL',
 'CRIME',
 'POLITICS',
 'POLITICS',
 'WELLNESS',
 'ENTERTAINMENT',
 'WELLNESS',
 'WORLDPOST',
 'FOOD 

In [45]:
compute_classification_report(y_test , df2_pred5)

                precision    recall  f1-score   support

ARTS & CULTURE       1.00      1.00      1.00         1
  BLACK VOICES       0.00      0.00      0.00         2
      BUSINESS       1.00      0.33      0.50         3
       COLLEGE       0.00      0.00      0.00         0
        COMEDY       0.00      0.00      0.00         2
         CRIME       0.50      1.00      0.67         1
CULTURE & ARTS       0.50      1.00      0.67         1
       DIVORCE       1.00      1.00      1.00         2
     EDUCATION       0.00      0.00      0.00         0
 ENTERTAINMENT       0.57      0.50      0.53         8
   ENVIRONMENT       1.00      1.00      1.00         1
         FIFTY       0.00      0.00      0.00         1
  FOOD & DRINK       1.00      1.00      1.00         3
     GOOD NEWS       0.00      0.00      0.00         1
         GREEN       0.00      0.00      0.00         1
HEALTHY LIVING       1.00      0.33      0.50         3
 HOME & LIVING       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

0.5

# **6**

In [46]:
df2_pred6 = find_top_categories_for_lists(doc_prb , 6)


In [47]:
df2_pred6

[['IMPACT', 'TRAVEL', 'WELLNESS', 'SPORTS', 'LATINO VOICES', 'WOMEN'],
 ['WORLDPOST', 'IMPACT', 'THE WORLDPOST', 'MONEY', 'WEIRD NEWS', 'TASTE'],
 ['POLITICS',
  'WEIRD NEWS',
  'IMPACT',
  'U.S. NEWS',
  'QUEER VOICES',
  'BLACK VOICES'],
 ['WOMEN', 'WEIRD NEWS', 'IMPACT', 'WORLDPOST', 'CRIME', 'EDUCATION'],
 ['IMPACT', 'MONEY', 'BUSINESS', 'ENVIRONMENT', 'WORLD NEWS', 'U.S. NEWS'],
 ['ARTS',
  'CULTURE & ARTS',
  'ARTS & CULTURE',
  'MEDIA',
  'IMPACT',
  'ENTERTAINMENT'],
 ['COMEDY', 'WOMEN', 'WEIRD NEWS', 'ENTERTAINMENT', 'MEDIA', 'IMPACT'],
 ['IMPACT', 'SPORTS', 'COLLEGE', 'QUEER VOICES', 'U.S. NEWS', 'FIFTY'],
 ['MONEY', 'TASTE', 'FOOD & DRINK', 'IMPACT', 'FIFTY', 'GREEN'],
 ['IMPACT', 'CRIME', 'POLITICS', 'U.S. NEWS', 'PARENTING', 'WEIRD NEWS'],
 ['ENTERTAINMENT', 'FOOD & DRINK', 'MEDIA', 'ARTS', 'IMPACT', 'TASTE'],
 ['MEDIA', 'WORLDPOST', 'THE WORLDPOST', 'U.S. NEWS', 'IMPACT', 'CRIME'],
 ['MEDIA', 'TECH', 'IMPACT', 'DIVORCE', 'BLACK VOICES', 'FIFTY'],
 ['IMPACT', 'WEIRD NEWS',

In [48]:
df2_pred6 = find_matching_category_for_each_list(df2_pred6 , y_test)


In [49]:
df2_pred6

['IMPACT',
 'WORLDPOST',
 'POLITICS',
 'WOMEN',
 'IMPACT',
 'CULTURE & ARTS',
 'ENTERTAINMENT',
 'IMPACT',
 'FOOD & DRINK',
 'POLITICS',
 'TASTE',
 'MEDIA',
 'MEDIA',
 'POLITICS',
 'TRAVEL',
 'WEIRD NEWS',
 'SCIENCE',
 'POLITICS',
 'BUSINESS',
 'IMPACT',
 'STYLE & BEAUTY',
 'HEALTHY LIVING',
 'POLITICS',
 'IMPACT',
 'MONEY',
 'FIFTY',
 'WEDDINGS',
 'IMPACT',
 'WORLDPOST',
 'IMPACT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'ENTERTAINMENT',
 'PARENTING',
 'ENTERTAINMENT',
 'DIVORCE',
 'LATINO VOICES',
 'COLLEGE',
 'POLITICS',
 'IMPACT',
 'IMPACT',
 'IMPACT',
 'TRAVEL',
 'FOOD & DRINK',
 'POLITICS',
 'IMPACT',
 'POLITICS',
 'RELIGION',
 'THE WORLDPOST',
 'ENVIRONMENT',
 'STYLE & BEAUTY',
 'ENTERTAINMENT',
 'STYLE & BEAUTY',
 'IMPACT',
 'IMPACT',
 'COMEDY',
 'IMPACT',
 'SPORTS',
 'IMPACT',
 'MONEY',
 'POLITICS',
 'IMPACT',
 'IMPACT',
 'WELLNESS',
 'TRAVEL',
 'CRIME',
 'POLITICS',
 'POLITICS',
 'WELLNESS',
 'ENTERTAINMENT',
 'WELLNESS',
 'WORLDPOST',
 'FOOD & DRINK

In [50]:
compute_classification_report(y_test , df2_pred6)

                precision    recall  f1-score   support

ARTS & CULTURE       1.00      1.00      1.00         1
  BLACK VOICES       0.00      0.00      0.00         2
      BUSINESS       1.00      0.33      0.50         3
       COLLEGE       0.00      0.00      0.00         0
        COMEDY       0.00      0.00      0.00         2
         CRIME       0.50      1.00      0.67         1
CULTURE & ARTS       0.50      1.00      0.67         1
       DIVORCE       1.00      1.00      1.00         2
     EDUCATION       0.00      0.00      0.00         0
 ENTERTAINMENT       0.67      0.50      0.57         8
   ENVIRONMENT       1.00      1.00      1.00         1
         FIFTY       0.00      0.00      0.00         1
  FOOD & DRINK       1.00      1.00      1.00         3
     GOOD NEWS       0.00      0.00      0.00         1
         GREEN       0.00      0.00      0.00         1
HEALTHY LIVING       1.00      0.33      0.50         3
 HOME & LIVING       0.00      0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

0.52

# ----------------------------------

In [52]:
df2 = df


In [53]:
# Create a new column "new_column" and set a specific value for matching rows
df2['new_category'] = 'Private Value'
mask = (df2["category"] == "ARTS") | (df2["category"] == "ARTS & CULTURE") | (df2["category"] == "CULTURE & ARTS")
df2.loc[mask, 'new_category'] = 'ARTS'
df2["new_category"].value_counts()

Private Value    186549
ARTS               3265
Name: new_category, dtype: int64

In [54]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "BLACK VOICES") | (df2["category"] == "QUEER VOICES") | (df2["category"] == "LATINO VOICES")
df2.loc[mask, 'new_category'] = 'VOICES'
df2["new_category"].value_counts()

Private Value    175747
VOICES            10802
ARTS               3265
Name: new_category, dtype: int64

In [55]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "BUSINESS") | (df2["category"] == "MONEY")
df2.loc[mask, 'new_category'] = 'BUSINESS'
df2["new_category"].value_counts()

Private Value    168860
VOICES            10802
BUSINESS           6887
ARTS               3265
Name: new_category, dtype: int64

In [56]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "COLLEGE") | (df2["category"] == "EDUCATION")
df2.loc[mask, 'new_category'] = 'EDUCATION'
df2["new_category"].value_counts()

Private Value    167037
VOICES            10802
BUSINESS           6887
ARTS               3265
EDUCATION          1823
Name: new_category, dtype: int64

In [57]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "DIVORCE") | (df2["category"] == "WEDDINGS") | (df2["category"] == "WOMEN")
df2.loc[mask, 'new_category'] = 'WOMEN'
df2["new_category"].value_counts()

Private Value    156774
VOICES            10802
WOMEN             10263
BUSINESS           6887
ARTS               3265
EDUCATION          1823
Name: new_category, dtype: int64

In [58]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "ENTERTAINMENT") | (df2["category"] == "COMEDY") | (df2["category"] == "MEDIA")
df2.loc[mask, 'new_category'] = 'ENTERTAINMENT'
df2["new_category"].value_counts()

Private Value    134954
ENTERTAINMENT     21820
VOICES            10802
WOMEN             10263
BUSINESS           6887
ARTS               3265
EDUCATION          1823
Name: new_category, dtype: int64

In [59]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "ENVIRONMENT") | (df2["category"] == "GREEN")
df2.loc[mask, 'new_category'] = 'ENVIRONMENT'
df2["new_category"].value_counts()

Private Value    131466
ENTERTAINMENT     21820
VOICES            10802
WOMEN             10263
BUSINESS           6887
ENVIRONMENT        3488
ARTS               3265
EDUCATION          1823
Name: new_category, dtype: int64

In [60]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "FOOD & DRINK") | (df2["category"] == "TASTE")
df2.loc[mask, 'new_category'] = 'FOOD'
df2["new_category"].value_counts()

Private Value    123195
ENTERTAINMENT     21820
VOICES            10802
WOMEN             10263
FOOD               8271
BUSINESS           6887
ENVIRONMENT        3488
ARTS               3265
EDUCATION          1823
Name: new_category, dtype: int64

In [61]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "HEALTHY LIVING") | (df2["category"] == "WELLNESS")
df2.loc[mask, 'new_category'] = 'WELLNESS'
df2["new_category"].value_counts()

Private Value    99987
WELLNESS         23208
ENTERTAINMENT    21820
VOICES           10802
WOMEN            10263
FOOD              8271
BUSINESS          6887
ENVIRONMENT       3488
ARTS              3265
EDUCATION         1823
Name: new_category, dtype: int64

In [62]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "PARENTING") | (df2["category"] == "PARENTS")
df2.loc[mask, 'new_category'] = 'PARENTING'
df2["new_category"].value_counts()

Private Value    87642
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
VOICES           10802
WOMEN            10263
FOOD              8271
BUSINESS          6887
ENVIRONMENT       3488
ARTS              3265
EDUCATION         1823
Name: new_category, dtype: int64

In [63]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "STYLE & BEAUTY") | (df2["category"] == "STYLE")
df2.loc[mask, 'new_category'] = 'STYLE'
df2["new_category"].value_counts()

Private Value    76273
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
BUSINESS          6887
ENVIRONMENT       3488
ARTS              3265
EDUCATION         1823
Name: new_category, dtype: int64

In [64]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "THE WORLDPOST") | (df2["category"] == "WORLDPOST") | (df2["category"] == "WORLD NEWS")
df2.loc[mask, 'new_category'] = 'WORLD NEWS'
df2["new_category"].value_counts()

Private Value    68070
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
ENVIRONMENT       3488
ARTS              3265
EDUCATION         1823
Name: new_category, dtype: int64

In [65]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "GOOD NEWS")
df2.loc[mask, 'new_category'] = 'GOOD NEWS'
df2["new_category"].value_counts()

Private Value    67031
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
ENVIRONMENT       3488
ARTS              3265
EDUCATION         1823
GOOD NEWS         1039
Name: new_category, dtype: int64

In [66]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "HOME & LIVING")
df2.loc[mask, 'new_category'] = 'HOME & LIVING'
df2["new_category"].value_counts()

Private Value    62714
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
EDUCATION         1823
GOOD NEWS         1039
Name: new_category, dtype: int64

In [67]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "POLITICS")
df2.loc[mask, 'new_category'] = 'POLITICS'
df2["new_category"].value_counts()

POLITICS         32441
Private Value    30273
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
EDUCATION         1823
GOOD NEWS         1039
Name: new_category, dtype: int64

In [68]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "RELIGION")
df2.loc[mask, 'new_category'] = 'RELIGION'
df2["new_category"].value_counts()

POLITICS         32441
Private Value    28394
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
RELIGION          1879
EDUCATION         1823
GOOD NEWS         1039
Name: new_category, dtype: int64

In [69]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "SCIENCE")
df2.loc[mask, 'new_category'] = 'SCIENCE'
df2["new_category"].value_counts()

POLITICS         32441
Private Value    26591
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
RELIGION          1879
EDUCATION         1823
SCIENCE           1803
GOOD NEWS         1039
Name: new_category, dtype: int64

In [70]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "SPORTS")
df2.loc[mask, 'new_category'] = 'SPORTS'
df2["new_category"].value_counts()

POLITICS         32441
WELLNESS         23208
Private Value    22177
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
SPORTS            4414
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
RELIGION          1879
EDUCATION         1823
SCIENCE           1803
GOOD NEWS         1039
Name: new_category, dtype: int64

In [71]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "TECH")
df2.loc[mask, 'new_category'] = 'TECH'
df2["new_category"].value_counts()

POLITICS         32441
WELLNESS         23208
ENTERTAINMENT    21820
Private Value    20074
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
SPORTS            4414
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
TECH              2103
RELIGION          1879
EDUCATION         1823
SCIENCE           1803
GOOD NEWS         1039
Name: new_category, dtype: int64

In [72]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "TRAVEL")
df2.loc[mask, 'new_category'] = 'TRAVEL'
df2["new_category"].value_counts()

POLITICS         32441
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
Private Value    10653
WOMEN            10263
TRAVEL            9421
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
SPORTS            4414
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
TECH              2103
RELIGION          1879
EDUCATION         1823
SCIENCE           1803
GOOD NEWS         1039
Name: new_category, dtype: int64

In [73]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "U.S. NEWS")
df2.loc[mask, 'new_category'] = 'U.S. NEWS'
df2["new_category"].value_counts()

POLITICS         32441
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
TRAVEL            9421
Private Value     9276
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
SPORTS            4414
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
TECH              2103
RELIGION          1879
EDUCATION         1823
SCIENCE           1803
U.S. NEWS         1377
GOOD NEWS         1039
Name: new_category, dtype: int64

In [74]:
# Create a new column "new_column" and set a specific value for matching rows
mask = (df2["category"] == "CRIME")
df2.loc[mask, 'new_category'] = 'CRIME'
df2["new_category"].value_counts()

POLITICS         32441
WELLNESS         23208
ENTERTAINMENT    21820
PARENTING        12345
STYLE            11369
VOICES           10802
WOMEN            10263
TRAVEL            9421
FOOD              8271
WORLD NEWS        8203
BUSINESS          6887
Private Value     6444
SPORTS            4414
HOME & LIVING     4317
ENVIRONMENT       3488
ARTS              3265
CRIME             2832
TECH              2103
RELIGION          1879
EDUCATION         1823
SCIENCE           1803
U.S. NEWS         1377
GOOD NEWS         1039
Name: new_category, dtype: int64

In [75]:
values_to_drop = ['WEIRD NEWS', 'FIFTY', 'IMPACT']

# Create a boolean mask to identify rows to drop
mask = df2['category'].isin(values_to_drop)

# Use the boolean mask to drop rows from the DataFrame
df2 = df2[~mask]
df2

,Unnamed: 0,link,headline,category,short_description,authors,date,new_category
0,0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,U.S. NEWS
1,1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,U.S. NEWS
2,2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,ENTERTAINMENT
3,3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,PARENTING
4,4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,U.S. NEWS
...,...,...,...,...,...,...,...,...
209521,209521,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28,TECH
209522,209522,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",NaN,2012-01-28,SPORTS
209523,209523,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",NaN,2012-01-28,SPORTS
209524,209524,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,NaN,2012-01-28,SPORTS


In [76]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183370 entries, 0 to 209525
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         183370 non-null  int64 
 1   link               183370 non-null  object
 2   headline           183369 non-null  object
 3   category           183370 non-null  object
 4   short_description  183370 non-null  object
 5   authors            150907 non-null  object
 6   date               183370 non-null  object
 7   new_category       183370 non-null  object
dtypes: int64(1), object(7)
memory usage: 12.6+ MB


In [77]:
df2.dropna(subset=['short_description'], inplace=True)

<ipython-input-77-a2301f4bb87c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(subset=['short_description'], inplace=True)


In [78]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183370 entries, 0 to 209525
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         183370 non-null  int64 
 1   link               183370 non-null  object
 2   headline           183369 non-null  object
 3   category           183370 non-null  object
 4   short_description  183370 non-null  object
 5   authors            150907 non-null  object
 6   date               183370 non-null  object
 7   new_category       183370 non-null  object
dtypes: int64(1), object(7)
memory usage: 12.6+ MB


In [79]:
X = df2.short_description
Y = df2.new_category

In [80]:
np.unique(Y)

array(['ARTS', 'BUSINESS', 'CRIME', 'EDUCATION', 'ENTERTAINMENT',
       'ENVIRONMENT', 'FOOD', 'GOOD NEWS', 'HOME & LIVING', 'PARENTING',
       'POLITICS', 'RELIGION', 'SCIENCE', 'SPORTS', 'STYLE', 'TECH',
       'TRAVEL', 'U.S. NEWS', 'VOICES', 'WELLNESS', 'WOMEN', 'WORLD NEWS'],
      dtype=object)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X,Y , shuffle=True, test_size = 100, random_state = 42 , stratify=Y)

In [82]:
np.unique(y_test)

array(['ARTS', 'BUSINESS', 'CRIME', 'EDUCATION', 'ENTERTAINMENT',
       'ENVIRONMENT', 'FOOD', 'GOOD NEWS', 'HOME & LIVING', 'PARENTING',
       'POLITICS', 'RELIGION', 'SCIENCE', 'SPORTS', 'STYLE', 'TECH',
       'TRAVEL', 'U.S. NEWS', 'VOICES', 'WELLNESS', 'WOMEN', 'WORLD NEWS'],
      dtype=object)

In [83]:
zsl = ZeroShotClassifier()
labels=['ARTS', 'BUSINESS', 'CRIME', 'EDUCATION', 'ENTERTAINMENT',
       'ENVIRONMENT', 'FOOD', 'GOOD NEWS', 'HOME & LIVING', 'PARENTING',
       'POLITICS', 'RELIGION', 'SCIENCE', 'SPORTS', 'STYLE', 'TECH',
       'TRAVEL', 'U.S. NEWS', 'VOICES', 'WELLNESS', 'WOMEN', 'WORLD NEWS']

In [84]:
%%time
doc_prb = zsl.predict(X_test.values, labels=labels, include_labels=True , batch_size = 16)

CPU times: user 20.4 s, sys: 71.5 ms, total: 20.4 s
Wall time: 20.7 s


In [85]:
doc_prb

[[('ARTS', 0.11789703369140625),
  ('BUSINESS', 0.0002366879489272833),
  ('CRIME', 0.16344182193279266),
  ('EDUCATION', 0.0006336911465041339),
  ('ENTERTAINMENT', 0.001455538673326373),
  ('ENVIRONMENT', 0.000962486898060888),
  ('FOOD', 0.000272720935754478),
  ('GOOD NEWS', 0.00015247229021042585),
  ('HOME & LIVING', 0.00025089210248552263),
  ('PARENTING', 0.0005714191938750446),
  ('POLITICS', 0.0007514490280300379),
  ('RELIGION', 0.0006179750780574977),
  ('SCIENCE', 0.00024406643933616579),
  ('SPORTS', 0.0003650837461464107),
  ('STYLE', 0.0007806693902239203),
  ('TECH', 0.000686793471686542),
  ('TRAVEL', 0.0004930737777613103),
  ('U.S. NEWS', 0.003084455383941531),
  ('VOICES', 0.0023671414237469435),
  ('WELLNESS', 0.00019589716976042837),
  ('WOMEN', 0.03394124284386635),
  ('WORLD NEWS', 0.002258125925436616)],
 [('ARTS', 0.00573867978528142),
  ('BUSINESS', 0.0006073769764043391),
  ('CRIME', 0.0007900056662037969),
  ('EDUCATION', 0.0003667690616566688),
  ('ENTERT

# **1**

In [86]:
df2_pred1 = find_top_categories_for_lists(doc_prb , 1)
# y_trn_pred = find_max_categories_for_lists(y_trn_prb)
# y_tst_pred = find_max_categories_for_lists(y_tst_prb)

In [87]:
df2_pred1

[['CRIME'],
 ['STYLE'],
 ['BUSINESS'],
 ['U.S. NEWS'],
 ['WELLNESS'],
 ['TRAVEL'],
 ['U.S. NEWS'],
 ['POLITICS'],
 ['ENTERTAINMENT'],
 ['ENTERTAINMENT'],
 ['STYLE'],
 ['FOOD'],
 ['SCIENCE'],
 ['GOOD NEWS'],
 ['ENTERTAINMENT'],
 ['ENTERTAINMENT'],
 ['PARENTING'],
 ['TRAVEL'],
 ['VOICES'],
 ['ENTERTAINMENT'],
 ['WOMEN'],
 ['ENTERTAINMENT'],
 ['U.S. NEWS'],
 ['WOMEN'],
 ['GOOD NEWS'],
 ['POLITICS'],
 ['TRAVEL'],
 ['SCIENCE'],
 ['BUSINESS'],
 ['WELLNESS'],
 ['POLITICS'],
 ['EDUCATION'],
 ['ENTERTAINMENT'],
 ['U.S. NEWS'],
 ['VOICES'],
 ['ENTERTAINMENT'],
 ['U.S. NEWS'],
 ['CRIME'],
 ['RELIGION'],
 ['U.S. NEWS'],
 ['VOICES'],
 ['HOME & LIVING'],
 ['WOMEN'],
 ['WELLNESS'],
 ['STYLE'],
 ['ENTERTAINMENT'],
 ['FOOD'],
 ['VOICES'],
 ['CRIME'],
 ['U.S. NEWS'],
 ['POLITICS'],
 ['WELLNESS'],
 ['PARENTING'],
 ['WOMEN'],
 ['ENTERTAINMENT'],
 ['CRIME'],
 ['POLITICS'],
 ['CRIME'],
 ['ENTERTAINMENT'],
 ['SPORTS'],
 ['STYLE'],
 ['POLITICS'],
 ['TRAVEL'],
 ['SCIENCE'],
 ['FOOD'],
 ['TECH'],
 ['TRAVEL'],
 

In [88]:
compute_classification_report(y_test , df2_pred1)

               precision    recall  f1-score   support

         ARTS       0.00      0.00      0.00         2
     BUSINESS       0.00      0.00      0.00         4
        CRIME       0.12      1.00      0.22         1
    EDUCATION       0.00      0.00      0.00         1
ENTERTAINMENT       0.28      0.42      0.33        12
  ENVIRONMENT       0.00      0.00      0.00         2
         FOOD       0.75      0.75      0.75         4
    GOOD NEWS       0.00      0.00      0.00         1
HOME & LIVING       0.00      0.00      0.00         2
    PARENTING       0.50      0.14      0.22         7
     POLITICS       0.67      0.33      0.44        18
     RELIGION       0.00      0.00      0.00         1
      SCIENCE       0.33      1.00      0.50         1
       SPORTS       1.00      0.50      0.67         2
        STYLE       0.75      0.50      0.60         6
         TECH       0.00      0.00      0.00         1
       TRAVEL       0.57      0.80      0.67         5
    U.S. 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.35

In [89]:
# compute_classification_report(y_trn , y_trn_pred)

In [90]:
# compute_classification_report(y_tst , y_tst_pred)

# **2**

## After getting the 2 maximum probability categories i check if **the first or the second** category is the same as true label and get the classification report

In [91]:
df2_pred2 = find_top_categories_for_lists(doc_prb , 2)
# y_trn_pred = find_max_2_categories_for_lists(y_trn_prb)
# y_tst_pred = find_max_2_categories_for_lists(y_tst_prb)

In [92]:
df2_pred2

[['CRIME', 'ARTS'],
 ['STYLE', 'U.S. NEWS'],
 ['BUSINESS', 'POLITICS'],
 ['U.S. NEWS', 'WORLD NEWS'],
 ['WELLNESS', 'HOME & LIVING'],
 ['TRAVEL', 'WORLD NEWS'],
 ['U.S. NEWS', 'ENVIRONMENT'],
 ['POLITICS', 'U.S. NEWS'],
 ['ENTERTAINMENT', 'ARTS'],
 ['ENTERTAINMENT', 'ARTS'],
 ['STYLE', 'FOOD'],
 ['FOOD', 'STYLE'],
 ['SCIENCE', 'ENVIRONMENT'],
 ['GOOD NEWS', 'U.S. NEWS'],
 ['ENTERTAINMENT', 'CRIME'],
 ['ENTERTAINMENT', 'ARTS'],
 ['PARENTING', 'VOICES'],
 ['TRAVEL', 'U.S. NEWS'],
 ['VOICES', 'TECH'],
 ['ENTERTAINMENT', 'BUSINESS'],
 ['WOMEN', 'GOOD NEWS'],
 ['ENTERTAINMENT', 'U.S. NEWS'],
 ['U.S. NEWS', 'POLITICS'],
 ['WOMEN', 'PARENTING'],
 ['GOOD NEWS', 'BUSINESS'],
 ['POLITICS', 'U.S. NEWS'],
 ['TRAVEL', 'GOOD NEWS'],
 ['SCIENCE', 'TECH'],
 ['BUSINESS', 'WELLNESS'],
 ['WELLNESS', 'SCIENCE'],
 ['POLITICS', 'TRAVEL'],
 ['EDUCATION', 'PARENTING'],
 ['ENTERTAINMENT', 'TRAVEL'],
 ['U.S. NEWS', 'ENVIRONMENT'],
 ['VOICES', 'ENTERTAINMENT'],
 ['ENTERTAINMENT', 'TECH'],
 ['U.S. NEWS', 'WOMEN']

In [93]:
df2_pred2 = find_matching_category_for_each_list(df2_pred2 , y_test)
# y_trn_pred = return_best_category_from_2_categories(y_trn , y_trn_pred)
# y_tst_pred = return_best_category_from_2_categories(y_tst , y_tst_pred)

In [94]:
df2_pred2

['ARTS',
 'STYLE',
 'POLITICS',
 'U.S. NEWS',
 'WELLNESS',
 'TRAVEL',
 'U.S. NEWS',
 'POLITICS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'STYLE',
 'FOOD',
 'SCIENCE',
 'GOOD NEWS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'VOICES',
 'TRAVEL',
 'VOICES',
 'ENTERTAINMENT',
 'WOMEN',
 'ENTERTAINMENT',
 'POLITICS',
 'WOMEN',
 'GOOD NEWS',
 'POLITICS',
 'TRAVEL',
 'SCIENCE',
 'BUSINESS',
 'WELLNESS',
 'POLITICS',
 'PARENTING',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'CRIME',
 'RELIGION',
 'U.S. NEWS',
 'RELIGION',
 'HOME & LIVING',
 'WOMEN',
 'WELLNESS',
 'STYLE',
 'TRAVEL',
 'FOOD',
 'VOICES',
 'WORLD NEWS',
 'U.S. NEWS',
 'POLITICS',
 'WELLNESS',
 'PARENTING',
 'WOMEN',
 'ENTERTAINMENT',
 'CRIME',
 'POLITICS',
 'CRIME',
 'ENTERTAINMENT',
 'SPORTS',
 'STYLE',
 'POLITICS',
 'TRAVEL',
 'SCIENCE',
 'FOOD',
 'TECH',
 'TRAVEL',
 'WOMEN',
 'CRIME',
 'U.S. NEWS',
 'GOOD NEWS',
 'POLITICS',
 'VOICES',
 'TECH',
 'RELIGION',
 'HOME & LIVING',
 'POLITICS',
 'SPORTS

In [95]:
compute_classification_report(y_test , df2_pred2)

               precision    recall  f1-score   support

         ARTS       0.50      0.50      0.50         2
     BUSINESS       0.33      0.25      0.29         4
        CRIME       0.17      1.00      0.29         1
    EDUCATION       0.00      0.00      0.00         1
ENTERTAINMENT       0.38      0.50      0.43        12
  ENVIRONMENT       0.00      0.00      0.00         2
         FOOD       0.80      1.00      0.89         4
    GOOD NEWS       0.00      0.00      0.00         1
HOME & LIVING       0.00      0.00      0.00         2
    PARENTING       1.00      0.29      0.44         7
     POLITICS       0.82      0.50      0.62        18
     RELIGION       0.33      1.00      0.50         1
      SCIENCE       0.33      1.00      0.50         1
       SPORTS       1.00      1.00      1.00         2
        STYLE       0.75      0.50      0.60         6
         TECH       0.50      1.00      0.67         1
       TRAVEL       0.62      1.00      0.77         5
    U.S. 

0.5

In [96]:
# compute_classification_report(y_trn , y_trn_pred)

In [97]:
# compute_classification_report(y_tst , y_tst_pred)

# **3**

In [98]:
df2_pred3 = find_top_categories_for_lists(doc_prb , 3)


In [99]:
df2_pred3

[['CRIME', 'ARTS', 'WOMEN'],
 ['STYLE', 'U.S. NEWS', 'WOMEN'],
 ['BUSINESS', 'POLITICS', 'U.S. NEWS'],
 ['U.S. NEWS', 'WORLD NEWS', 'GOOD NEWS'],
 ['WELLNESS', 'HOME & LIVING', 'GOOD NEWS'],
 ['TRAVEL', 'WORLD NEWS', 'ENTERTAINMENT'],
 ['U.S. NEWS', 'ENVIRONMENT', 'CRIME'],
 ['POLITICS', 'U.S. NEWS', 'VOICES'],
 ['ENTERTAINMENT', 'ARTS', 'HOME & LIVING'],
 ['ENTERTAINMENT', 'ARTS', 'GOOD NEWS'],
 ['STYLE', 'FOOD', 'ENVIRONMENT'],
 ['FOOD', 'STYLE', 'ARTS'],
 ['SCIENCE', 'ENVIRONMENT', 'GOOD NEWS'],
 ['GOOD NEWS', 'U.S. NEWS', 'VOICES'],
 ['ENTERTAINMENT', 'CRIME', 'U.S. NEWS'],
 ['ENTERTAINMENT', 'ARTS', 'STYLE'],
 ['PARENTING', 'VOICES', 'U.S. NEWS'],
 ['TRAVEL', 'U.S. NEWS', 'TECH'],
 ['VOICES', 'TECH', 'U.S. NEWS'],
 ['ENTERTAINMENT', 'BUSINESS', 'ARTS'],
 ['WOMEN', 'GOOD NEWS', 'VOICES'],
 ['ENTERTAINMENT', 'U.S. NEWS', 'CRIME'],
 ['U.S. NEWS', 'POLITICS', 'VOICES'],
 ['WOMEN', 'PARENTING', 'CRIME'],
 ['GOOD NEWS', 'BUSINESS', 'U.S. NEWS'],
 ['POLITICS', 'U.S. NEWS', 'ARTS'],
 ['TR

In [100]:
df2_pred3 = find_matching_category_for_each_list(df2_pred3 , y_test)


In [101]:
df2_pred3

['ARTS',
 'STYLE',
 'POLITICS',
 'U.S. NEWS',
 'WELLNESS',
 'TRAVEL',
 'U.S. NEWS',
 'POLITICS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'STYLE',
 'FOOD',
 'SCIENCE',
 'VOICES',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'VOICES',
 'TRAVEL',
 'VOICES',
 'ENTERTAINMENT',
 'WOMEN',
 'ENTERTAINMENT',
 'POLITICS',
 'WOMEN',
 'GOOD NEWS',
 'POLITICS',
 'TRAVEL',
 'SCIENCE',
 'BUSINESS',
 'WELLNESS',
 'POLITICS',
 'PARENTING',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'CRIME',
 'WORLD NEWS',
 'U.S. NEWS',
 'RELIGION',
 'HOME & LIVING',
 'WOMEN',
 'WELLNESS',
 'STYLE',
 'TRAVEL',
 'FOOD',
 'VOICES',
 'WORLD NEWS',
 'POLITICS',
 'POLITICS',
 'WELLNESS',
 'PARENTING',
 'WOMEN',
 'ENTERTAINMENT',
 'CRIME',
 'POLITICS',
 'CRIME',
 'ENTERTAINMENT',
 'SPORTS',
 'STYLE',
 'POLITICS',
 'TRAVEL',
 'SCIENCE',
 'FOOD',
 'TECH',
 'TRAVEL',
 'WOMEN',
 'VOICES',
 'U.S. NEWS',
 'GOOD NEWS',
 'POLITICS',
 'VOICES',
 'TECH',
 'RELIGION',
 'HOME & LIVING',
 'POLITICS',
 'SPORTS'

In [102]:
compute_classification_report(y_test , df2_pred3)

               precision    recall  f1-score   support

         ARTS       0.67      1.00      0.80         2
     BUSINESS       0.33      0.25      0.29         4
        CRIME       0.20      1.00      0.33         1
    EDUCATION       0.00      0.00      0.00         1
ENTERTAINMENT       0.43      0.50      0.46        12
  ENVIRONMENT       0.00      0.00      0.00         2
         FOOD       0.80      1.00      0.89         4
    GOOD NEWS       0.00      0.00      0.00         1
HOME & LIVING       0.00      0.00      0.00         2
    PARENTING       1.00      0.29      0.44         7
     POLITICS       0.83      0.56      0.67        18
     RELIGION       0.50      1.00      0.67         1
      SCIENCE       0.33      1.00      0.50         1
       SPORTS       1.00      1.00      1.00         2
        STYLE       0.80      0.67      0.73         6
         TECH       0.50      1.00      0.67         1
       TRAVEL       0.62      1.00      0.77         5
    U.S. 

0.56

# **4**

In [103]:
df2_pred4 = find_top_categories_for_lists(doc_prb , 4)


In [104]:
df2_pred4

[['CRIME', 'ARTS', 'WOMEN', 'U.S. NEWS'],
 ['STYLE', 'U.S. NEWS', 'WOMEN', 'WORLD NEWS'],
 ['BUSINESS', 'POLITICS', 'U.S. NEWS', 'VOICES'],
 ['U.S. NEWS', 'WORLD NEWS', 'GOOD NEWS', 'CRIME'],
 ['WELLNESS', 'HOME & LIVING', 'GOOD NEWS', 'FOOD'],
 ['TRAVEL', 'WORLD NEWS', 'ENTERTAINMENT', 'GOOD NEWS'],
 ['U.S. NEWS', 'ENVIRONMENT', 'CRIME', 'TRAVEL'],
 ['POLITICS', 'U.S. NEWS', 'VOICES', 'CRIME'],
 ['ENTERTAINMENT', 'ARTS', 'HOME & LIVING', 'WELLNESS'],
 ['ENTERTAINMENT', 'ARTS', 'GOOD NEWS', 'U.S. NEWS'],
 ['STYLE', 'FOOD', 'ENVIRONMENT', 'HOME & LIVING'],
 ['FOOD', 'STYLE', 'ARTS', 'WELLNESS'],
 ['SCIENCE', 'ENVIRONMENT', 'GOOD NEWS', 'WORLD NEWS'],
 ['GOOD NEWS', 'U.S. NEWS', 'VOICES', 'WELLNESS'],
 ['ENTERTAINMENT', 'CRIME', 'U.S. NEWS', 'POLITICS'],
 ['ENTERTAINMENT', 'ARTS', 'STYLE', 'TRAVEL'],
 ['PARENTING', 'VOICES', 'U.S. NEWS', 'GOOD NEWS'],
 ['TRAVEL', 'U.S. NEWS', 'TECH', 'ARTS'],
 ['VOICES', 'TECH', 'U.S. NEWS', 'ENVIRONMENT'],
 ['ENTERTAINMENT', 'BUSINESS', 'ARTS', 'STYLE']

In [105]:
df2_pred4 = find_matching_category_for_each_list(df2_pred4 , y_test)


In [106]:
df2_pred4

['ARTS',
 'STYLE',
 'POLITICS',
 'U.S. NEWS',
 'WELLNESS',
 'TRAVEL',
 'U.S. NEWS',
 'POLITICS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'HOME & LIVING',
 'FOOD',
 'SCIENCE',
 'VOICES',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'VOICES',
 'TRAVEL',
 'VOICES',
 'ENTERTAINMENT',
 'WOMEN',
 'ENTERTAINMENT',
 'POLITICS',
 'WOMEN',
 'GOOD NEWS',
 'ENTERTAINMENT',
 'TRAVEL',
 'SCIENCE',
 'BUSINESS',
 'WELLNESS',
 'POLITICS',
 'PARENTING',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'CRIME',
 'WORLD NEWS',
 'U.S. NEWS',
 'RELIGION',
 'HOME & LIVING',
 'WOMEN',
 'WELLNESS',
 'STYLE',
 'TRAVEL',
 'FOOD',
 'VOICES',
 'WORLD NEWS',
 'POLITICS',
 'POLITICS',
 'WELLNESS',
 'PARENTING',
 'WOMEN',
 'ENTERTAINMENT',
 'POLITICS',
 'POLITICS',
 'CRIME',
 'ENTERTAINMENT',
 'SPORTS',
 'STYLE',
 'POLITICS',
 'TRAVEL',
 'SCIENCE',
 'FOOD',
 'TECH',
 'WOMEN',
 'WELLNESS',
 'VOICES',
 'U.S. NEWS',
 'GOOD NEWS',
 'POLITICS',
 'VOICES',
 'TECH',
 'RELIGION',
 'PARENTING',
 'POLITI

In [107]:
compute_classification_report(y_test , df2_pred4)

               precision    recall  f1-score   support

         ARTS       0.67      1.00      0.80         2
     BUSINESS       0.33      0.25      0.29         4
        CRIME       0.33      1.00      0.50         1
    EDUCATION       0.00      0.00      0.00         1
ENTERTAINMENT       0.47      0.58      0.52        12
  ENVIRONMENT       0.00      0.00      0.00         2
         FOOD       0.80      1.00      0.89         4
    GOOD NEWS       0.00      0.00      0.00         1
HOME & LIVING       0.50      0.50      0.50         2
    PARENTING       1.00      0.43      0.60         7
     POLITICS       0.92      0.61      0.73        18
     RELIGION       0.50      1.00      0.67         1
      SCIENCE       0.33      1.00      0.50         1
       SPORTS       1.00      1.00      1.00         2
        STYLE       1.00      0.67      0.80         6
         TECH       0.50      1.00      0.67         1
       TRAVEL       0.71      1.00      0.83         5
    U.S. 

0.63

# **5**

In [108]:
df2_pred5 = find_top_categories_for_lists(doc_prb , 5)


In [109]:
df2_pred5

[['CRIME', 'ARTS', 'WOMEN', 'U.S. NEWS', 'VOICES'],
 ['STYLE', 'U.S. NEWS', 'WOMEN', 'WORLD NEWS', 'GOOD NEWS'],
 ['BUSINESS', 'POLITICS', 'U.S. NEWS', 'VOICES', 'WORLD NEWS'],
 ['U.S. NEWS', 'WORLD NEWS', 'GOOD NEWS', 'CRIME', 'TECH'],
 ['WELLNESS', 'HOME & LIVING', 'GOOD NEWS', 'FOOD', 'VOICES'],
 ['TRAVEL', 'WORLD NEWS', 'ENTERTAINMENT', 'GOOD NEWS', 'TECH'],
 ['U.S. NEWS', 'ENVIRONMENT', 'CRIME', 'TRAVEL', 'WORLD NEWS'],
 ['POLITICS', 'U.S. NEWS', 'VOICES', 'CRIME', 'WORLD NEWS'],
 ['ENTERTAINMENT', 'ARTS', 'HOME & LIVING', 'WELLNESS', 'VOICES'],
 ['ENTERTAINMENT', 'ARTS', 'GOOD NEWS', 'U.S. NEWS', 'VOICES'],
 ['STYLE', 'FOOD', 'ENVIRONMENT', 'HOME & LIVING', 'U.S. NEWS'],
 ['FOOD', 'STYLE', 'ARTS', 'WELLNESS', 'BUSINESS'],
 ['SCIENCE', 'ENVIRONMENT', 'GOOD NEWS', 'WORLD NEWS', 'WELLNESS'],
 ['GOOD NEWS', 'U.S. NEWS', 'VOICES', 'WELLNESS', 'WORLD NEWS'],
 ['ENTERTAINMENT', 'CRIME', 'U.S. NEWS', 'POLITICS', 'BUSINESS'],
 ['ENTERTAINMENT', 'ARTS', 'STYLE', 'TRAVEL', 'U.S. NEWS'],
 ['

In [110]:
df2_pred5 = find_matching_category_for_each_list(df2_pred5 , y_test)


In [111]:
df2_pred5

['ARTS',
 'STYLE',
 'POLITICS',
 'U.S. NEWS',
 'WELLNESS',
 'TRAVEL',
 'U.S. NEWS',
 'POLITICS',
 'VOICES',
 'VOICES',
 'HOME & LIVING',
 'FOOD',
 'WELLNESS',
 'VOICES',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'VOICES',
 'TRAVEL',
 'VOICES',
 'ENTERTAINMENT',
 'WOMEN',
 'ENTERTAINMENT',
 'POLITICS',
 'WOMEN',
 'GOOD NEWS',
 'ENTERTAINMENT',
 'TRAVEL',
 'SCIENCE',
 'WORLD NEWS',
 'WELLNESS',
 'POLITICS',
 'PARENTING',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'CRIME',
 'WORLD NEWS',
 'U.S. NEWS',
 'RELIGION',
 'HOME & LIVING',
 'WOMEN',
 'WELLNESS',
 'STYLE',
 'TRAVEL',
 'FOOD',
 'VOICES',
 'WORLD NEWS',
 'POLITICS',
 'POLITICS',
 'WELLNESS',
 'PARENTING',
 'WOMEN',
 'ENTERTAINMENT',
 'POLITICS',
 'POLITICS',
 'POLITICS',
 'ENTERTAINMENT',
 'SPORTS',
 'STYLE',
 'POLITICS',
 'TRAVEL',
 'SCIENCE',
 'FOOD',
 'TECH',
 'WOMEN',
 'WELLNESS',
 'VOICES',
 'U.S. NEWS',
 'GOOD NEWS',
 'BUSINESS',
 'VOICES',
 'TECH',
 'RELIGION',
 'PARENTING',
 'POLITICS',
 'S

In [112]:
compute_classification_report(y_test , df2_pred5)

               precision    recall  f1-score   support

         ARTS       0.67      1.00      0.80         2
     BUSINESS       0.67      0.50      0.57         4
        CRIME       0.50      1.00      0.67         1
    EDUCATION       0.00      0.00      0.00         1
ENTERTAINMENT       0.54      0.58      0.56        12
  ENVIRONMENT       0.00      0.00      0.00         2
         FOOD       0.80      1.00      0.89         4
    GOOD NEWS       0.33      1.00      0.50         1
HOME & LIVING       0.50      0.50      0.50         2
    PARENTING       1.00      0.43      0.60         7
     POLITICS       1.00      0.67      0.80        18
     RELIGION       0.50      1.00      0.67         1
      SCIENCE       0.50      1.00      0.67         1
       SPORTS       1.00      1.00      1.00         2
        STYLE       1.00      0.67      0.80         6
         TECH       0.50      1.00      0.67         1
       TRAVEL       0.71      1.00      0.83         5
    U.S. 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.7

# **6**

In [113]:
df2_pred6 = find_top_categories_for_lists(doc_prb , 6)


In [114]:
df2_pred6

[['CRIME', 'ARTS', 'WOMEN', 'U.S. NEWS', 'VOICES', 'WORLD NEWS'],
 ['STYLE', 'U.S. NEWS', 'WOMEN', 'WORLD NEWS', 'GOOD NEWS', 'ARTS'],
 ['BUSINESS', 'POLITICS', 'U.S. NEWS', 'VOICES', 'WORLD NEWS', 'TECH'],
 ['U.S. NEWS', 'WORLD NEWS', 'GOOD NEWS', 'CRIME', 'TECH', 'BUSINESS'],
 ['WELLNESS', 'HOME & LIVING', 'GOOD NEWS', 'FOOD', 'VOICES', 'ENVIRONMENT'],
 ['TRAVEL', 'WORLD NEWS', 'ENTERTAINMENT', 'GOOD NEWS', 'TECH', 'STYLE'],
 ['U.S. NEWS', 'ENVIRONMENT', 'CRIME', 'TRAVEL', 'WORLD NEWS', 'ARTS'],
 ['POLITICS', 'U.S. NEWS', 'VOICES', 'CRIME', 'WORLD NEWS', 'STYLE'],
 ['ENTERTAINMENT', 'ARTS', 'HOME & LIVING', 'WELLNESS', 'VOICES', 'STYLE'],
 ['ENTERTAINMENT', 'ARTS', 'GOOD NEWS', 'U.S. NEWS', 'VOICES', 'WELLNESS'],
 ['STYLE', 'FOOD', 'ENVIRONMENT', 'HOME & LIVING', 'U.S. NEWS', 'WELLNESS'],
 ['FOOD', 'STYLE', 'ARTS', 'WELLNESS', 'BUSINESS', 'U.S. NEWS'],
 ['SCIENCE', 'ENVIRONMENT', 'GOOD NEWS', 'WORLD NEWS', 'WELLNESS', 'TECH'],
 ['GOOD NEWS', 'U.S. NEWS', 'VOICES', 'WELLNESS', 'WORLD 

In [115]:
df2_pred6 = find_matching_category_for_each_list(df2_pred6 , y_test)


In [116]:
df2_pred6

['ARTS',
 'STYLE',
 'POLITICS',
 'U.S. NEWS',
 'WELLNESS',
 'TRAVEL',
 'U.S. NEWS',
 'POLITICS',
 'VOICES',
 'VOICES',
 'HOME & LIVING',
 'FOOD',
 'WELLNESS',
 'VOICES',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'VOICES',
 'TRAVEL',
 'VOICES',
 'ENTERTAINMENT',
 'WOMEN',
 'ENTERTAINMENT',
 'POLITICS',
 'WOMEN',
 'GOOD NEWS',
 'ENTERTAINMENT',
 'TRAVEL',
 'SCIENCE',
 'WORLD NEWS',
 'WELLNESS',
 'POLITICS',
 'PARENTING',
 'BUSINESS',
 'U.S. NEWS',
 'ENTERTAINMENT',
 'ENTERTAINMENT',
 'U.S. NEWS',
 'CRIME',
 'WORLD NEWS',
 'U.S. NEWS',
 'RELIGION',
 'HOME & LIVING',
 'WOMEN',
 'WELLNESS',
 'STYLE',
 'TRAVEL',
 'FOOD',
 'WELLNESS',
 'WORLD NEWS',
 'POLITICS',
 'POLITICS',
 'WELLNESS',
 'PARENTING',
 'WOMEN',
 'ENTERTAINMENT',
 'POLITICS',
 'POLITICS',
 'POLITICS',
 'ENTERTAINMENT',
 'SPORTS',
 'STYLE',
 'POLITICS',
 'TRAVEL',
 'SCIENCE',
 'FOOD',
 'TECH',
 'WOMEN',
 'WELLNESS',
 'VOICES',
 'U.S. NEWS',
 'GOOD NEWS',
 'BUSINESS',
 'VOICES',
 'TECH',
 'RELIGION',
 'PARENTING',
 'POLITICS',
 'SPOR

In [117]:
compute_classification_report(y_test , df2_pred6)

               precision    recall  f1-score   support

         ARTS       0.67      1.00      0.80         2
     BUSINESS       1.00      0.75      0.86         4
        CRIME       0.50      1.00      0.67         1
    EDUCATION       0.00      0.00      0.00         1
ENTERTAINMENT       0.67      0.67      0.67        12
  ENVIRONMENT       1.00      0.50      0.67         2
         FOOD       0.80      1.00      0.89         4
    GOOD NEWS       0.33      1.00      0.50         1
HOME & LIVING       0.50      0.50      0.50         2
    PARENTING       1.00      0.43      0.60         7
     POLITICS       1.00      0.67      0.80        18
     RELIGION       0.50      1.00      0.67         1
      SCIENCE       0.50      1.00      0.67         1
       SPORTS       1.00      1.00      1.00         2
        STYLE       1.00      0.67      0.80         6
         TECH       0.50      1.00      0.67         1
       TRAVEL       0.83      1.00      0.91         5
    U.S. 

0.75